- Project filter CDWM
- Source: "Cascaded Mach-Zehnder wavelength filters in silicon photonics for low loss and flat pass-band WDM (de-)multiplexing,"

In [1]:
import sys, os, imp
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c

C:\Users\alves\AppData\Local\Temp\ipykernel_24408\2964610077.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import sys, os, imp


In [2]:
lumapiFile = "/opt/lumerical/v221/api/python/lumapi.py"
lumapiFile = "C:\\Program Files\\Lumerical\\v221\\api\\python\\lumapi.py"
lumapi = imp.load_source("lumapi", lumapiFile)
interApi = lumapi.INTERCONNECT(hide = False)

In [3]:
interApi.expressioncheckoff()

In [4]:
# units
um = 1e-6
nm = 1e-9

central_wavelength = 1550*nm

# index
neff = 2.3531702
ng = 4.3458796

wvlngth_start = 1.500 * um
wvlngth_stop = 1.600 * um

number_of_points = 50e3

In [ ]:
interApi.switchtolayout()
interApi.select('ONA')
interApi.delete()

# ONA

interApi.addelement('Optical Network Analyzer')
interApi.set('name', 'ONA')
interApi.set('number of input ports', 2)
interApi.set('input parameter', 2)
interApi.set('start frequency', c/wvlngth_start)
interApi.set('stop frequency', c/wvlngth_stop)
interApi.set('number of points', number_of_points)

interApi.setposition('ONA', 0, -400)


In [ ]:
def filter_1st(pw_coupling, L, delta_L, name_prefix, position_y): 
    interApi.switchtolayout()


    names = {
        'coupler_1': f'{name_prefix}_coupler_1',
        'coupler_2': f'{name_prefix}_coupler_2',
        'wg_1': f'{name_prefix}_wg_1',
        'wg_2': f'{name_prefix}_wg_2'
    }

     # Remove elementos existentes com os mesmos nomes
    for element in names.values():
        interApi.select(element)
        interApi.delete()

    # Adiciona elementos com nomes exclusivos
    interApi.addelement('Waveguide Coupler')
    interApi.set('name', names['coupler_1'])

    interApi.addelement('Waveguide Coupler')
    interApi.set('name', names['coupler_2'])

    interApi.addelement('Straight Waveguide')
    interApi.set('name', names['wg_1'])
    interApi.set('frequency', (c/(central_wavelength)))
    interApi.set('length', L)
    interApi.set('effective index 1', neff)
    interApi.set('group index 1', ng)

    interApi.addelement('Straight Waveguide')
    interApi.set('name', names['wg_2'])
    interApi.set('frequency', (c/(central_wavelength)))
    interApi.set('length', L + delta_L)
    interApi.set('effective index 1', neff)
    interApi.set('group index 1', ng)

    #positions
    interApi.setposition(names['coupler_1'], -250, 50 + position_y)
    interApi.setposition(names['coupler_2'], 250, 50 + position_y)
    interApi.setposition(names['wg_1'], 0,-100 + position_y)
    interApi.setposition(names['wg_2'], 0, 100 + position_y)

    # connection

    interApi.connect("ONA", "output", "coupler_1", "port 1")
    interApi.connect("coupler_1", "port 3", "wg_1", "port 1")
    interApi.connect("coupler_1", "port 4", "wg_2", "port 1")
    interApi.connect("wg_1", "port 2", "coupler_2", "port 1")
    interApi.connect("wg_2", "port 2", "coupler_2", "port 2")
    interApi.connect("coupler_2", "port 3", "ONA", "input 1")
    interApi.connect("coupler_2", "port 4", "ONA", "input 2")

    return names  # retorna os nomes dos elementos para controle externo

In [28]:
filter_1 = filter_1st(0.5, 2*um, 10*um, 'filter1', 0)

In [19]:
print(filter_1['coupler_1'])

filter1_coupler_1
